In [1]:
"""
@author: ogouvert

Variational Inference algorithm for Ordinal Non-Negative Matrix Factorization (OrdNMF)

- DCPF MODEL:
W ~ Gamma(aphaW,betaW)      ## UxK (size of W)
H ~ Gamma(aphaH,betaH)      ## IxK
C ~ OrdNMF(W*H)             ## UxI

- VARIATIONAL INFERENCE:
p(W,H,C,N) \approx q(C|N)q(N)q(W)q(H)
where:
    q(W) = Gamma()
    q(H) = Gamma()
    q(C|N) = Mult()
    q(N) = ZTP()
"""

#%% Ordinal NMF

import numpy as np
import scipy.special as special
import scipy.sparse as sparse
import os
import time
import pickle as pickle
import sys

In [2]:
def stat_gamma(shape, rate):
    """
    Statistic of a gamma distribution:
        x \sim Gamma(shape, rate)
        INPUT: shape and rate parameters
        OUTPUT: E[x], E[log(x)], H the entropy
    """
    E = shape / rate
    dig_shape = special.digamma(shape)
    Elog = dig_shape - np.log(rate)
    entropy = shape - np.log(rate) + special.gammaln(shape) + (1 - shape) * dig_shape
    return E, Elog, entropy
  
def gamma_elbo(shape, rate, Ex, Elogx):
    """ Part of the ELBO linked to the gamma prior """
    gammaln = special.gammaln(shape)
    return (shape - 1) * Elogx - rate * Ex + shape * np.log(rate) - gammaln

def q_Gamma(shape, _shape, rate, _rate):
    """ Calculate both statistic and ELBO """
    E, Elog, entropy = stat_gamma(shape + _shape, rate + _rate)
    elbo = gamma_elbo(shape, rate, E, Elog)
    elbo = elbo.sum() + entropy.sum()
    return E, Elog, elbo

def Ord_generate(L, theta):
    Y = np.zeros(L.shape)
    X = L / n p.random.gamma(1, 1, L.shape)
    for t in theta:
        Y = Y + (X > 1. / t)
    Y = Y.astype(int)
    return Y
    
def _writeline_and_time(s):
    sys.stdout.write(s)
    sys.stdout.flush()
    return time.time()


In [3]:
class OrdNMF():
    def __init__(self, K,
                 alphaW = 1., alphaH = 1., betaW=1., betaH = 1.):
        """
        K (int) - number of latent factors
        alphaW (float, >0) - shape parameter of the prior of W
        alphaH (float, >0) - shape parameter of the prior of H
        betaW (float, >0) - rate parameter of the prior of W
        betaH (float, >0) - rate parameter of the prior of H
        """
        self.K = K
        self.alphaW = alphaW
        self.alphaH = alphaH
        self.betaW = betaW
        self.betaH = betaH
        self.score={}
        self.classname = 'OrdNMF'
        # Save arg
        saved_args_init = locals()
        saved_args_init.pop('self')
        self.saved_args_init = saved_args_init
        
    def fit(self, Y, T, 
            seed=None, 
            opt_hyper = ['beta'],
            approx = False,
            precision = 10**(-5),
            max_iter = 10**5,
            min_iter = 0,
            verbose = False,
            save = True,
            save_dir = '',
            prefix = None,
            suffix = None):
        """
        ------- INPUT VARIABLES -------
        Y (sparse matrix of size UxI, type:int) - Observed data, values from 0 to T
        T - maximum value in Y
        
        ------- OPTIONAL VARIABLES -------
        seed (int)
        opt_hyper (list of float)
            'beta' - update the scale parameters of the gamma prior of W and H
                    betaW of size U, betaH of size I
            'betaH' - update the scale parameters of the gamma prior of H
                    betaH is a scalar
        approx (bool) - if True, the variable N is approximated by a dirac located in 1
        precision (float) - stopping criterion on the ELBO
        max_iter (int) - maximum iteration number
        min_iter (int) - minimum iteration number 
        save (bool) - Saving the final class object
        save_dir (str) - Path of the saved file
        prefix, suffix (str) - prefix and suffix to use in the name of the saved file
        
        ------- SAVED VARIABLES -------
        Ew, Elogw : Expectations: Ew = E[W] and Elogw = E[log(W)]
        Eh, Elogh : idem for variable H
        Elbo : Evolution of the ELBO
        """
        self.seed = seed
        np.random.seed(seed)
        self.T = T
        self.opt_hyper = opt_hyper
        self.approx = approx
        self.verbose = verbose
        self.precision = precision
        # Save
        self.save = save
        self.save_dir = save_dir
        self.prefix = prefix
        self.suffix = suffix
        self.filename = self.filename(prefix, suffix)
        # Save arg
        saved_args_fit = locals()
        saved_args_fit.pop('self')
        saved_args_fit.pop('Y')
        self.saved_args_fit = saved_args_fit
        # Timer
        start_time = time.time()
                
        # Shape
        U,I = Y.shape
        u,i = Y.nonzero()
        y = Y.data
        # Init - matrix companion
        delta = self.init_delta(Y)  #delta = np.ones(T+1); delta[0]=0;
        H = (np.triu(np.ones((T+1,T+1))).dot(delta[:,np.newaxis]))[:,0] 
        theta0 = H[0]
        G = theta0 - H
        Gy = sparse.csr_matrix((G[y],(u,i)), shape=(U,I))
        # Init - W & H
        Ew = np.random.gamma(1.,1.,(U,self.K))
        Eh = np.random.gamma(1.,1.,(I,self.K))
        s_wh = np.dot(np.sum(Ew,0,keepdims=True),np.sum(Eh,0,keepdims=True).T)[0,0]
        
        # Local
        Sw, Sh, En, elboLoc = self.q_loc(Y,delta,Ew,Eh)
            
        self.Elbo = [-float("inf")]
        self.info = []
        for n in range(max_iter):
            # Time
            if verbose:
                print('ITERATION #%d' % n)
                start_t = _writeline_and_time('\tUpdates...')
            # Hyper parameter
            if np.isin('beta',opt_hyper):
                self.betaW = self.alphaW/Ew.mean(axis=1,keepdims=True)
                self.betaH = self.alphaH/Eh.mean(axis=1,keepdims=True)
            if np.isin('betaH',opt_hyper):
                self.betaH = self.alphaH / np.mean(Eh)
            # Updates Delta
            lbd = np.sum(Ew[u,:]*Eh[i,:],1)
            S_lbd = s_wh
            for l in range(T,0,-1): # {T,...,1}
                S_lbd = S_lbd - np.sum(lbd[Y.data==l+1])
                delta[l] = np.sum(En[Y.data==l])/S_lbd
            H = (np.triu(np.ones((T+1,T+1))).dot(delta[:,np.newaxis]))[:,0] 
            theta0 = H[0]
            G = theta0 - H
            Gy = sparse.csr_matrix((G[y],(u,i)), shape=(U,I))
            # Global updates
            Ew, Elogw, elboW = q_Gamma(self.alphaW , Sw, 
                                       self.betaW, theta0*np.sum(Eh,0,keepdims=True) - Gy.dot(Eh))
            Eh, Elogh, elboH = q_Gamma(self.alphaH, Sh,
                                       self.betaH, theta0*np.sum(Ew,0,keepdims=True) - Gy.T.dot(Ew))
            s_wh = np.dot(np.sum(Ew,0,keepdims=True),np.sum(Eh,0,keepdims=True).T)[0,0]
            # Local updates
            Sw, Sh, En, elboLoc = self.q_loc(Y,delta,np.exp(Elogw),np.exp(Elogh))
            # Elbo update
            elbo = elboLoc - theta0*s_wh + np.sum(Ew*Gy.dot(Eh)) + elboW + elboH
            if n==0:
                self.rate = float('inf')
            else:
                self.rate = (elbo-self.Elbo[-1])/np.abs(self.Elbo[-1])
            if verbose:
                print('\r\tUpdates: time=%.2f'% (time.time() - start_t))
                print('\tRate:' + str(self.rate))
            if elbo<self.Elbo[-1]:
                self.Elbo.append(elbo) 
                raise ValueError('Elbo diminue!')
            if np.isnan(elbo):
                #pass
                raise ValueError('elbo NAN')
            elif self.rate<precision and n>=min_iter:
                self.Elbo.append(elbo) 
                break
            self.Elbo.append(elbo) 
            self.info.append(delta.copy())
        
        self.delta = delta
        self.theta = (np.triu(np.ones((T+1,T+1)),1).dot(delta[:,np.newaxis]))[:,0]
        self.Ew = Ew.copy()
        self.Eh = Eh.copy()
        self.En = En.copy()
        self.Elogw = Elogw.copy()
        self.Elogh = Elogh.copy()
        
        self.duration = time.time()-start_time
        
        # Save
        if self.save:
            self.save_model()
     
    def init_delta(self,Y):
        """ Initialization of delta w.r.t. the histogram values of Y  """
        hist_values = np.bincount(Y.data)
        hist_values[0] = Y.nnz
        cum_hist = np.cumsum(hist_values, dtype=float)
        delta = hist_values/cum_hist
        delta[0]=0
        return delta

    def q_loc(self,Y,delta,W,H):
        """ 
        q(C,N) = q(N)q(C|N)
        q(C|N) = Multinomial
        q(N) = ZTP
        
        OUTPUT:
        en - data of the sparse matrix En
        Sw = \sum_i E[c_{uik}]
        Sh = \sum_u E[c_{uik}]
        """
        # Product        
        u,i = Y.nonzero()
        Lbd = np.sum(W[u,:]*H[i,:],1)
        delta_y = delta[Y.data]
        # En
        if self.approx == False:
            en = Lbd * delta_y / (1. - np.exp(-Lbd * delta_y))
            en[np.isnan(en)] = 1.
        else :
            en = np.ones_like(Lbd)
        # Sum C
        R = sparse.csr_matrix((en / Lbd, (u, i)), shape = Y.shape) # UxI
        Sw = W*(R.dot(H)) 
        Sh = H*(R.T.dot(W))
        # ELBO
        elbo = np.sum(np.log(np.expm1(Lbd * delta_y)))
        return Sw, Sh, en, elbo

    def filename(self,prefix,suffix):
        if prefix is not None:
            prefix = prefix+'_'
        else:
            prefix = ''
        if suffix is not None:
            suffix = '_'+suffix
        else:
            suffix = ''
        return prefix + self.classname + \
                '_K%d' % (self.K) + \
                '_T%d' % (self.T) + \
                '_alpha%.2f_%.2f' %(self.alphaW, self.alphaH) + \
                '_beta%.2f_%.2f' % (self.betaW, self.betaH) + \
                '_opthyper_' + '_'.join(sorted(self.opt_hyper)) + \
                '_approxN_' + str(self.approx) + \
                '_tol%.1e' %(self.precision) + \
                '_seed' + str(self.seed) + suffix
            
    def save_model(self):
        with open(os.path.join(self.save_dir, self.filename), 'wb') as handle:
                pickle.dump(self, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    def copy_attributes(self,oobj):
        self.__dict__ = oobj.__dict__.copy()

In [4]:
"""
@author: ogouvert
"""

#%% 
import sys
sys.path.append("../../model/OrdNMF")
sys.path.append("../../function")

import os
import pickle as pickle

from OrdNMF import OrdNMF

import preprocess_data  as prep

#%% 
prop_test = 0.2
seed_test = 213

with open('../../data/ML/ml_145_U1.99e+04_I7.80e+03_min_uc20_sc20', 'rb') as f:
    out = pickle.load(f)
    Y = out['Y_listen']
U,I = Y.shape

#%% Directory
Y_train,Y_test = prep.divide_train_test(Y,prop_test=prop_test,seed=seed_test)
save_dir = 'out/seed_%d' %(seed_test)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
        
#%% IN 
opt_hyper = ['beta']  

Ks = [25,50,100,150,200,250]
Seeds = range(5) # Seed of the different initializations
tol = 10**(-5)
min_iter = 0
max_iter = 10**5
    
#%% Ord NMF
alpha = .3 

# Ordinal
if True:
    for approx in [False]: 
        for K in Ks:
            for seed in Seeds:
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(Y_train, T=Y_train.max(), 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML', 
                          verbose=True)
   
# Binary
if True:
    for approx in [True,False]: # Approx Bernoulli -> Poisson
        for K in Ks:
            for seed in Seeds:
                R1 = Y_train>=1
                R1.eliminate_zeros()
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(R1, T=1, 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML_geq1', 
                          verbose=False)
        
if True:
    for approx in [True,False]: # Approx Bernoulli -> Poisson
        for K in Ks:
            for seed in Seeds:                
                R8 = Y_train>=8
                R8.eliminate_zeros()
                model = OrdNMF(K=K, alphaW=alpha, alphaH=alpha)
                model.fit(R8, T=1, 
                          seed=seed, opt_hyper=opt_hyper, 
                          approx = approx, 
                          precision=tol, min_iter=min_iter, max_iter=max_iter,
                          save=True, save_dir=save_dir,prefix='ML_geq8', 
                          verbose=False)
        

ITERATION #0
	Updates: time=1.40
	Rate:inf
ITERATION #1
	Updates: time=1.42
	Rate:0.1554315175666314
ITERATION #2
	Updates: time=1.39
	Rate:0.0034385195626273264
ITERATION #3
	Updates: time=1.48
	Rate:0.003120615627033682
ITERATION #4
	Updates: time=1.72
	Rate:0.004479919548619812
ITERATION #5
	Updates: time=1.45
	Rate:0.0067934034666450906
ITERATION #6
	Updates: time=1.65
	Rate:0.00980879173806608
ITERATION #7
	Updates: time=1.62
	Rate:0.01256377258924036
ITERATION #8
	Updates: time=1.78
	Rate:0.013858874404044418
ITERATION #9
	Updates: time=1.85
	Rate:0.013319940186837548
ITERATION #10
	Updates: time=1.95
	Rate:0.011663903213096086
ITERATION #11
	Updates: time=2.09
	Rate:0.009889353500155705
ITERATION #12
	Updates: time=5.22
	Rate:0.008519574553748601
ITERATION #13
	Updates: time=6.13
	Rate:0.007541826205244988
ITERATION #14
	Updates: time=3.98
	Rate:0.006757310026604353
ITERATION #15
	Updates: time=1.68
	Rate:0.006031101693720052
ITERATION #16
	Updates: time=1.68
	Rate:0.00530935170

	Updates: time=2.87
	Rate:2.0074933628862187e-05
ITERATION #133
	Updates: time=2.66
	Rate:1.9759544764703964e-05
ITERATION #134
	Updates: time=2.73
	Rate:1.9497849313630926e-05
ITERATION #135
	Updates: time=2.51
	Rate:1.9317609392578034e-05
ITERATION #136
	Updates: time=2.81
	Rate:1.920322228402312e-05
ITERATION #137
	Updates: time=2.64
	Rate:1.910552803263611e-05
ITERATION #138
	Updates: time=2.70
	Rate:1.895841355624624e-05
ITERATION #139
	Updates: time=3.11
	Rate:1.870098337973449e-05
ITERATION #140
	Updates: time=3.21
	Rate:1.8315537333113425e-05
ITERATION #141
	Updates: time=4.75
	Rate:1.7840911196594765e-05
ITERATION #142
	Updates: time=4.26
	Rate:1.7349247265370837e-05
ITERATION #143
	Updates: time=4.53
	Rate:1.6894856245399942e-05
ITERATION #144
	Updates: time=4.48
	Rate:1.6494706689784082e-05
ITERATION #145
	Updates: time=9.83
	Rate:1.6150056091222012e-05
ITERATION #146
	Updates: time=2.88
	Rate:1.5856405943839485e-05
ITERATION #147
	Updates: time=2.63
	Rate:1.5606579756637296

	Updates: time=1.90
	Rate:4.179727181705267e-05
ITERATION #82
	Updates: time=1.97
	Rate:4.009026118007745e-05
ITERATION #83
	Updates: time=1.83
	Rate:3.8382184910585554e-05
ITERATION #84
	Updates: time=1.90
	Rate:3.6748972767314e-05
ITERATION #85
	Updates: time=1.96
	Rate:3.530915982084252e-05
ITERATION #86
	Updates: time=2.00
	Rate:3.4096032712517055e-05
ITERATION #87
	Updates: time=2.10
	Rate:3.3050437279116986e-05
ITERATION #88
	Updates: time=4.83
	Rate:3.208294269056459e-05
ITERATION #89
	Updates: time=5.90
	Rate:3.113975112139065e-05
ITERATION #90
	Updates: time=4.87
	Rate:3.02057283562283e-05
ITERATION #91
	Updates: time=3.49
	Rate:2.9302224465965033e-05
ITERATION #92
	Updates: time=4.26
	Rate:2.8435539236586727e-05
ITERATION #93
	Updates: time=2.08
	Rate:2.756022635715044e-05
ITERATION #94
	Updates: time=2.08
	Rate:2.6636715959882896e-05
ITERATION #95
	Updates: time=1.99
	Rate:2.5654415766345436e-05
ITERATION #96
	Updates: time=1.94
	Rate:2.462182077954557e-05
ITERATION #97
	Upd

	Updates: time=5.12
	Rate:6.194355836218028e-05
ITERATION #86
	Updates: time=5.65
	Rate:6.024529805391255e-05
ITERATION #87
	Updates: time=3.31
	Rate:5.883122021869166e-05
ITERATION #88
	Updates: time=2.76
	Rate:5.764883465619877e-05
ITERATION #89
	Updates: time=2.82
	Rate:5.6606858229659465e-05
ITERATION #90
	Updates: time=2.40
	Rate:5.563081863919838e-05
ITERATION #91
	Updates: time=2.46
	Rate:5.469582915458834e-05
ITERATION #92
	Updates: time=2.10
	Rate:5.3822640122949277e-05
ITERATION #93
	Updates: time=2.05
	Rate:5.301130632791824e-05
ITERATION #94
	Updates: time=2.02
	Rate:5.218381091333725e-05
ITERATION #95
	Updates: time=2.21
	Rate:5.121220438989072e-05
ITERATION #96
	Updates: time=2.22
	Rate:5.000657965147276e-05
ITERATION #97
	Updates: time=2.17
	Rate:4.8541938520600335e-05
ITERATION #98
	Updates: time=2.24
	Rate:4.688619205882648e-05
ITERATION #99
	Updates: time=2.27
	Rate:4.517551982677383e-05
ITERATION #100
	Updates: time=2.33
	Rate:4.353351143943458e-05
ITERATION #101
	Up

	Updates: time=9.43
	Rate:1.1342906423332758e-05
ITERATION #215
	Updates: time=10.86
	Rate:1.1206105531957541e-05
ITERATION #216
	Updates: time=7.37
	Rate:1.1071467610134395e-05
ITERATION #217
	Updates: time=6.17
	Rate:1.0948033170734191e-05
ITERATION #218
	Updates: time=5.94
	Rate:1.083680501821838e-05
ITERATION #219
	Updates: time=5.96
	Rate:1.0728943885424715e-05
ITERATION #220
	Updates: time=6.05
	Rate:1.0605909671304706e-05
ITERATION #221
	Updates: time=5.40
	Rate:1.0457729316989232e-05
ITERATION #222
	Updates: time=4.95
	Rate:1.0295678577338253e-05
ITERATION #223
	Updates: time=4.87
	Rate:1.0141241982286451e-05
ITERATION #224
	Updates: time=5.32
	Rate:1.0012792917786792e-05
ITERATION #225
	Updates: time=4.98
	Rate:9.920194369140913e-06
ITERATION #0
	Updates: time=5.54
	Rate:inf
ITERATION #1
	Updates: time=5.13
	Rate:0.155405311323161
ITERATION #2
	Updates: time=5.11
	Rate:0.0033362956247449905
ITERATION #3
	Updates: time=5.88
	Rate:0.0029028127145227156
ITERATION #4
	Updates: tim

...

	Updates: time=8.81
	Rate:0.00011360088901520958
ITERATION #81
	Updates: time=8.32
	Rate:0.00011115392767672279
ITERATION #82
	Updates: time=8.63
	Rate:0.00010853590703948401
ITERATION #83
	Updates: time=8.48
	Rate:0.00010544326696865206
ITERATION #84
	Updates: time=8.06
	Rate:0.00010216228196901094
ITERATION #85
	Updates: time=8.69
	Rate:9.925362725081843e-05
ITERATION #86
	Updates: time=8.54
	Rate:9.600367734880702e-05
ITERATION #87
	Updates: time=6.13
	Rate:9.215481620440647e-05
ITERATION #88
	Updates: time=4.53
	Rate:8.852653493943141e-05
ITERATION #89
	Updates: time=4.10
	Rate:8.518512100446076e-05
ITERATION #90
	Updates: time=3.90
	Rate:8.178332602257133e-05
ITERATION #91
	Updates: time=4.08
	Rate:7.8313549767162e-05
ITERATION #92
	Updates: time=3.47
	Rate:7.52436505460324e-05
ITERATION #93
	Updates: time=3.42
	Rate:7.267827282431209e-05
ITERATION #94
	Updates: time=3.69
	Rate:7.034494948034218e-05
ITERATION #95
	Updates: time=4.04
	Rate:6.796282720529781e-05
ITERATION #96
	Updat

	Updates: time=2.82
	Rate:1.2333932613934877e-05
ITERATION #211
	Updates: time=2.72
	Rate:1.2112205498176202e-05
ITERATION #212
	Updates: time=2.95
	Rate:1.1905412814493052e-05
ITERATION #213
	Updates: time=2.89
	Rate:1.171297963300032e-05
ITERATION #214
	Updates: time=3.31
	Rate:1.151175728884188e-05
ITERATION #215
	Updates: time=3.41
	Rate:1.1272374313036634e-05
ITERATION #216
	Updates: time=3.81
	Rate:1.0974702027221524e-05
ITERATION #217
	Updates: time=3.88
	Rate:1.0620949074500486e-05
ITERATION #218
	Updates: time=3.94
	Rate:1.0255297627131346e-05
ITERATION #219
	Updates: time=4.70
	Rate:9.923760589504713e-06
ITERATION #0
	Updates: time=4.01
	Rate:inf
ITERATION #1
	Updates: time=4.02
	Rate:0.34156978725695414
ITERATION #2
	Updates: time=3.50
	Rate:0.0070585731586406165
ITERATION #3
	Updates: time=3.88
	Rate:0.0039627662247816746
ITERATION #4
	Updates: time=3.22
	Rate:0.005440642545796648
ITERATION #5
	Updates: time=3.04
	Rate:0.008168897455738212
ITERATION #6
	Updates: time=3.18
	

	Updates: time=4.13
	Rate:3.30502003541821e-05
ITERATION #123
	Updates: time=3.61
	Rate:3.2105786756518493e-05
ITERATION #124
	Updates: time=3.27
	Rate:3.128413452613368e-05
ITERATION #125
	Updates: time=3.56
	Rate:3.051831059797367e-05
ITERATION #126
	Updates: time=3.26
	Rate:2.977147556039842e-05
ITERATION #127
	Updates: time=3.32
	Rate:2.912549660007933e-05
ITERATION #128
	Updates: time=3.20
	Rate:2.866043359455008e-05
ITERATION #129
	Updates: time=3.15
	Rate:2.832617797364152e-05
ITERATION #130
	Updates: time=3.20
	Rate:2.802966537000349e-05
ITERATION #131
	Updates: time=3.25
	Rate:2.767959775577559e-05
ITERATION #132
	Updates: time=3.11
	Rate:2.7223116757491086e-05
ITERATION #133
	Updates: time=3.19
	Rate:2.6693902444782873e-05
ITERATION #134
	Updates: time=3.16
	Rate:2.6153561616936677e-05
ITERATION #135
	Updates: time=3.12
	Rate:2.5630854052938458e-05
ITERATION #136
	Updates: time=3.13
	Rate:2.5127798740084034e-05
ITERATION #137
	Updates: time=3.25
	Rate:2.4623559259726645e-05
I

	Updates: time=6.74
	Rate:0.0002963033814281146
ITERATION #46
	Updates: time=8.67
	Rate:0.00028238706934125734
ITERATION #47
	Updates: time=8.14
	Rate:0.000269165934988466
ITERATION #48
	Updates: time=8.23
	Rate:0.00025638341890217776
ITERATION #49
	Updates: time=8.43
	Rate:0.00024379558498983673
ITERATION #50
	Updates: time=8.31
	Rate:0.000231318015419272
ITERATION #51
	Updates: time=8.15
	Rate:0.00021947154558409879
ITERATION #52
	Updates: time=7.03
	Rate:0.00020874523371488155
ITERATION #53
	Updates: time=10.03
	Rate:0.00019927635831230162
ITERATION #54
	Updates: time=7.22
	Rate:0.00019093970412811722
ITERATION #55
	Updates: time=6.50
	Rate:0.00018353148882886317
ITERATION #56
	Updates: time=7.97
	Rate:0.00017681479289876774
ITERATION #57
	Updates: time=10.01
	Rate:0.00017072516946838642
ITERATION #58
	Updates: time=7.82
	Rate:0.00016539574075194398
ITERATION #59
	Updates: time=8.58
	Rate:0.0001606763554937789
ITERATION #60
	Updates: time=9.59
	Rate:0.00015617312086002623
ITERATION 

KeyboardInterrupt: 